In [ ]:
#import necessary modules
import numpy as np
import rasterio as rs
import matplotlib.pyplot as plt
import earthpy.spatial as es
import earthpy.plot as ep
from numpy import loadtxt
from osgeo import gdal, gdalconst
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.ensemble import IsolationForest
from sklearn import svm

In [ ]:
#Set imagery as variables
asi = #'Path_to_SingleDateImage_Amsterdam.tif'
ami = #'Path_to_MedianImage_Amsterdam.tif'
msi = #'Path_to_SingleDateImage_Milano.tif'
mmi = #'Path_to_MedianImage_Milano.tif'
bsi = #'Path_to_SingleDateImage_Budapest.tif'
bmi = #'Path_to_MedianImage_Budapest.tif'
filist = [asi, ami, msi, mmi, bsi, bmi]

In [ ]:
def read_data(inras):
    # Read data
    img = gdal.Open(inras, gdal.GA_ReadOnly) 
    bands = [img.GetRasterBand(i).ReadAsArray() for i in range(1, img.RasterCount + 1)]
    img = np.array(bands)
    img = img[0:12,:,:]
    img = np.transpose(img, [1, 2, 0])
    img_rgbnir = img[:,:,[3,2,1,7]]
    return img_rgbnir

#Import Feature extraction
NDWI = loadtxt(#'Path_to_NDWI.csv', delimiter=',')
NDVI = loadtxt(#'Path_to_NDVI.csv', delimiter=',')
EMP = np.load(#'Path_to_EMP.npy')

GLCM_load = loadtxt(#'Path_to_GLCM.csv', delimiter=',')
img = read_data(#imgvariable)
nr, nc, nb = img.shape
nf = 16
GLCM = GLCM_load.reshape(nr,nc,nf)
where_are_NaNs = isnan(GLCM)
GLCM[where_are_NaNs] = 0

In [ ]:
img = read_data(#imgvarable)
stack = np.concatenate((img, EMP, GLCM), axis=2)
stacked = np.dstack((stack, NDVI, NDWI))
#Setup for IF and OCSVM
X = stacked
Ytr1 = Twita
nr,nc,nb = X.shape
ns = nr*nc
X = X.reshape((ns,nb))
Ytr = Ytr1.reshape((ns,))
ind = np.where(Ytr > 0)
Xtr = X[ind[0],:]
Ytr = Ytr[ind[0]]
standard_scaler = StandardScaler()
Xtr = standard_scaler.fit_transform(Xtr) 
X = standard_scaler.transform(X)

In [ ]:
%%time
#IF
model = IsolationForest(n_estimators = 50)
model.fit(Xtr)
yhat = model.score_samples(X)
params = model.get_params(deep=True)
yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
class_map1 = np.reshape(yhat1,(nr,nc))

In [ ]:
%%time
#OCSVM
model = svm.OneClassSVM(kernel = 'rbf', nu = 0.5)
model.fit(Xtr)
yhat2 = model.score_samples(X)
yhat3 = (yhat2-min(yhat2))/(max(yhat2)-min(yhat2))
class_map2 = np.reshape(yhat3,(nr,nc))

In [ ]:
IsolationForest_img = np.savetxt(#'Path_to_IsolationForest_img.csv', class_map1, delimiter=',')
Load_IF_Result = np.loadtxt(#'Path_to_IsolationForest_img.csv', delimiter=',')

In [ ]:
OCSVM_img = np.savetxt(#'Path_to_OCSVM_img.csv', class_map2, delimiter=',')
Load_OCSVM_Result = np.loadtxt(#'Path_to_OCSVM_img.csv', delimiter=',')

In [ ]:
# Create classes and apply to NDVI results
if_class_bins = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0]
if_class = np.digitize(Load_IF_Result, if_class_bins)

# Apply the nodata mask to the newly classified NDVI data
if_landsat_class = np.ma.masked_where(
    np.ma.getmask(Load_IF_Result), if_class
)
np.unique(if_class)

# Define color map
#nbr_colors = ["gray", "y", "yellowgreen", "g", "darkgreen"]
#nbr_cmap = ListedColormap(nbr_colors)

# Define class names
if_cat_names = [
    "1.0 - 0.9",
    "0.9 - 0.8",
    "0.8 - 0.7",
    "0.7 - 0.6",
    "0.6 - 0.5",
    "0.5 - 0.4",
    "0.4 - 0.3",
    "0.3 - 0.2",
    "0.2 - 0.1",
    "0.1 - 0.0"
]

# Get list of classes
classes = np.unique(if_class)
classes = classes.tolist()
# The mask returns a value of none in the classes. remove that
classes = classes[0:10]

# Plot your data
fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(if_class, cmap="RdYlGn")

ep.draw_legend(im_ax=im, classes=classes, titles=if_cat_names)
#ax.set_title(
#    "SVM Anomaly score Amsterdam Single Image",
#    fontsize=20,
#)
ax.set_axis_off()

# Auto adjust subplot to fit figure size
plt.tight_layout()